# Setup
## External Dependencies

In [2]:
using Flux, Plots, Random, Statistics;

## Sample Preparation

In [4]:
function gen_real_samples(n=100)
    # Generate random inputs between -0.5 and 0.5
    X1 = rand(n) .- 0.5

    # compute their outputs
    X2 = X1.^2 
    X1 = reshape(X1, (n,1))
    X2 = reshape(X2, (n,1))
    X = [X1 X2]

    # include labels
    y = ones((n,1))

    return X, y
end

function generate_latent_points(latent_dim, n)
    x_input = rand(latent_dim*n)
    x_input = reshape(x_input, n, latent_dim)
    return x_input
end

function gen_fake_samples(generator, latent_dim, n=100)
    x_input = generate_latent_points(latent_dim, n)

    X = generator(x_input).data
    
    y = zeros((n,1))
    return X,y
end

n_inputs = 2;
n_outputs = 2;
latent_dim = 20;

# Architecture
## Generator

In [6]:
gen = Chain(
    x-> reshape(x, latent_dim, :),
    Dense(latent_dim, 15, relu),
    Dense(15, n_outputs, leakyrelu)
);

## Discriminator

In [7]:
disc = Chain(
    x-> reshape(x, n_inputs, :),
    Dense(n_inputs, 25, relu),
    Dense(25, 1, sigmoid)
);

## Optimizers

In [8]:
opt_gen  = ADAM(params(gen), 0.001f0, β1 = 0.5);
opt_disc = ADAM(params(disc), 0.001f0, β1 = 0.5);

┌ Warning: ADAM(params) is deprecated; use ADAM(η::Float64) instead
│   caller = ip:0x0
└ @ Core :-1


## Helper Functions

In [9]:
function nullify_grad!(p)
  if typeof(p) <: TrackedArray
    p.grad .= 0.0f0
  end
  return p
end

function zero_grad!(model)
  model = mapleaves(nullify_grad!, model)
end

zero_grad! (generic function with 1 method)

# Loss and Training
## Loss

In [10]:
# binary cross entropy
function bce(ŷ, y)
    neg_abs = -abs.(ŷ)
    mean(relu.(ŷ) .- ŷ .* y .+ log.(1 .+ exp.(neg_abs)))
end

bce (generic function with 1 method)

## Training

In [12]:
training_steps = 0
verbose_freq = 100

function train(x)
    global training_steps
  
    # z = rand(dist, noise_dim, BATCH_SIZE) |> gpu
    realX, realy = gen_real_samples(x)
   
    zero_grad!(disc)
   
    D_real = disc(realX)
    D_real_loss = bce(D_real, realy)
  
    fakeX, fakey = gen_fake_samples(gen, latent_dim, x)

    D_fake = disc(fakeX)
    D_fake_loss = bce(D_fake, fakey)
  
    D_loss = D_real_loss + D_fake_loss
  
    Flux.back!(D_loss)
    opt_disc()
  
    zero_grad!(gen)
    
    # z = rand(dist, noise_dim, BATCH_SIZE) |> gpu
    fakeX, fakey = gen_fake_samples(gen, latent_dim, x)
    D_fake = disc(fakeX)
    G_loss = bce(D_fake, ones(x))
  
    Flux.back!(G_loss)
    opt_gen()
  
    if training_steps % verbose_freq == 0
        println("D Loss: $(D_loss.data) | G loss: $(G_loss.data)")
    end
    
    training_steps += 1
    param(0.0f0)
end

# set to 10,000 if you have a few minutes to wait and see better results
NUM_EPOCHS = 1000;

"""
for e = 1:NUM_EPOCHS
    println("Epoch $e: ")
    train(500)
end
"""

UndefVarError: UndefVarError: e not defined

## Visualize

In [ ]:
test_disc = [[i, i^2] for i ∈ -0.5:0.01:0.5]
println("Average discrimenator correct guess: ", mean(disc.(test_disc)))

x_fake, y_fake = gen_fake_samples(gen, latent_dim)
scatter(x_fake[1, :], x_fake[2, :], title="GAN Attempt")